<a href="https://colab.research.google.com/github/alexfok/IntroDeepLearning2022/blob/main/IDL_Project_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Alexander Fok 308669944

Avi Dvir 204423735

Gal Cohen 204675805

# Project: Unsupervised Deep Embedding for Clustering Analysis

**Assignment Responsible**: May Malka.

In this project, we build a Unsupervised Deep Embedding for Clustering Analysis with AE, etc 


In [49]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

#!pip install leveldb
#!pip install features
!pip install scipy
import leveldb
import shutil
import random
#import features
from joblib import Parallel, delayed
#from skimage import feature
from skimage.feature import hog
import multiprocessing
#from scipy.misc import imrotate
import scipy

# Run NN and torch data on GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Notebook Control variables
# Print DEBUG info
DEBUG = False
# Variable controlling debug prints while training\infering with NN
DEBUG_NN = False
# Create the data sets or load the preloaded np arrays
CREATE_DATA = True
# Path to save the created numpy arrays
array_save_path = '/content/gdrive/My Drive/IntroDeepLearning2022Data/project/data.npy'
# Path to load the created numpy arrays
array_load_path = '/content/gdrive/My Drive/IntroDeepLearning2022Data/project/data.npy.npz'
best_model_path = '/content/gdrive/MyDrive/IntroDeepLearning2022Data/project/model_chkpt_epoch_20.pk'
checkpoint_path = '/content/gdrive/MyDrive/IntroDeepLearning2022Data/project/model_chkpt_epoch_{}.pk'

# Raw data set paths
path_train = "/content/gdrive/My Drive/IntroDeepLearning2022Data/project"
#path_train = "/content/gdrive/My Drive/IntroDeepLearning2022Data/data/train/*.*"
#path_test_m = "/content/gdrive/My Drive/IntroDeepLearning2022Data/data/test_m/*.*"
#path_test_w = "/content/gdrive/My Drive/IntroDeepLearning2022Data/data/test_w/*.*"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Data Load

Download the data from https://www.dropbox.com/s/6gdcpmfddojrl8o/data.rar?dl=0.

Unzip the file. There are three
main folders: `train`, `test_w` and `test_m`. Data in `train` will be used for
training and validation, and the data in the other folders will be used for testing.
This is so that the entire class will have the same test sets. The dataset is comprised of triplets of pairs, where each such triplet of image pairs was taken in a similar setting (by the same person).

We've separated `test_w` and `test_m` so that we can track our model performance 
for women's shoes and men's shoes separately. Each of the test sets contain images of either exclusively men's shoes or women's
shoes.

Upload this data to Google Colab.
Then, mount Google Drive from your Google Colab notebook:

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Part (a)


In [43]:
'''
!ls -laht *-ubyte
#!pwd
!cp train-labels-idx1-ubyte "$path_train"
!cp train-images-idx3-ubyte "$path_train"
!cp t10k-labels-idx1-ubyte "$path_train"
!cp t10k-images-idx3-ubyte "$path_train"
#!cp *-ubyte "$path_train"
#!cp ./*-ubyte "$path_train"/*

#!cp './train-images-idx3-ubyte' '$path_train'
#path_train='test'
!echo "$path_train"
'''
#!tar -xf stl10_binary.tar.gz
!ls -laht .
!ls -laht stl10_binary

total 2.6G
-rw-r--r-- 1 root  root 513K Dec 28 15:08 train_X.bin_org.jpg
drwxr-xr-x 1 root  root 4.0K Dec 28 13:26 .
drwx------ 6 root  root 4.0K Dec 28 12:10 gdrive
drwxr-xr-x 1 root  root 4.0K Dec 28 12:05 ..
drwxr-xr-x 1 root  root 4.0K Dec 20 20:19 sample_data
drwxr-xr-x 4 root  root 4.0K Dec 20 20:18 .config
-rw-r--r-- 1 root  root 2.5G Apr  8  2011 stl10_binary.tar.gz
drwxr-xr-x 2 10176 1000 4.0K Apr  8  2011 stl10_binary
-rw-r--r-- 1 root  root  59K Jul 21  2000 train-labels-idx1-ubyte
-rw-r--r-- 1 root  root  45M Jul 21  2000 train-images-idx3-ubyte
-rw-r--r-- 1 root  root 9.8K Jul 21  2000 t10k-labels-idx1-ubyte
-rw-r--r-- 1 root  root 7.5M Jul 21  2000 t10k-images-idx3-ubyte
total 3.0G
drwxr-xr-x 1 root  root  4.0K Dec 28 13:26 ..
drwxr-xr-x 2 10176  1000 4.0K Apr  8  2011 .
-rw-r--r-- 1 10176 users 7.9K Apr  8  2011 test_y.bin
-rw-r--r-- 1 10176 users 211M Apr  8  2011 test_X.bin
-rw-r--r-- 1 10176 users 132M Apr  8  2011 train_X.bin
-rw-r--r-- 1 10176 users 4.9K Apr  8  201

In [16]:
#!/bin/sh
# Download data set and save it on GDrive
# MNIST dataset
!wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!gunzip -f train-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!gunzip -f train-labels-idx1-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!gunzip -f t10k-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
!gunzip -f t10k-labels-idx1-ubyte.gz

#!cp ./*-ubyte "$path_train"
!cp train-labels-idx1-ubyte "$path_train"
!cp train-images-idx3-ubyte "$path_train"
!cp t10k-labels-idx1-ubyte "$path_train"
!cp t10k-images-idx3-ubyte "$path_train"

# STL dataset
!wget http://ai.stanford.edu/~acoates/stl10/stl10_binary.tar.gz
!tar -xf stl10_binary.tar.gz
!ls -la
#!cp './train-images-idx3-ubyte' "$path_train"


--2022-12-28 13:00:34--  http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 172.67.171.76, 104.21.29.36, 2606:4700:3036::ac43:ab4c, ...
Connecting to yann.lecun.com (yann.lecun.com)|172.67.171.76|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9912422 (9.5M) [application/x-gzip]
Saving to: ‘train-images-idx3-ubyte.gz’

train-images-idx3-u 100%[===================>]   9.45M  --.-KB/s    in 0.08s   

2022-12-28 13:00:34 (111 MB/s) - ‘train-images-idx3-ubyte.gz’ saved [9912422/9912422]

--2022-12-28 13:00:35--  http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 172.67.171.76, 104.21.29.36, 2606:4700:3036::ac43:ab4c, ...
Connecting to yann.lecun.com (yann.lecun.com)|172.67.171.76|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28881 (28K) [application/x-gzip]
Saving to: ‘train-labels-idx1-ubyte.gz’

train-labels-idx1-u 100%[==========

In [70]:
def dispImg(X, n, fname=None):
  h = X.shape[1]
  w = X.shape[2]
  c = X.shape[3]
  buff = np.zeros((n*w, n*w, c), dtype=np.uint8)

  for i in range(n):
    for j in range(n):
      buff[i*h:(i+1)*h, j*w:(j+1)*w, :] = X[i*n+j]

  if fname is None:
    cv2.imshow('a', buff)
    cv2.waitKey(0)
  else:
    cv2.imwrite(fname, buff)

def hog_picture(hog, resolution):
#    from scipy.misc import imrotate
    glyph1 = np.zeros((resolution, resolution), dtype=np.uint8)
    glyph1[:, round(resolution / 2)-1:round(resolution / 2) + 1] = 255
    glyph = np.zeros((resolution, resolution, 9), dtype=np.uint8)
    glyph[:, :, 0] = glyph1
    for i in range(1, 9):
        tt = scipy.ndimage.interpolation.rotate(glyph1, -i * 20)
        tt = cv2.resize(tt, (resolution, resolution), interpolation = cv2.INTER_AREA)
        glyph[:, :, i] = tt
#        glyph[:, :, i] = imrotate(glyph1, -i * 20)

#    print(f'tt.shape: {tt.shape}')
    print(f'hog.shape: {hog.shape}')
    shape = hog.shape
    clamped_hog = hog.copy()
    clamped_hog[hog < 0] = 0
    image = np.zeros((resolution * shape[0], resolution * shape[1]), dtype=np.float32)
    for i in range(shape[0]):
        for j in range(shape[1]):
            for k in range(9):
                image[i*resolution:(i+1)*resolution, j*resolution:(j+1)*resolution] = np.maximum(image[i*resolution:(i+1)*resolution, j*resolution:(j+1)*resolution], clamped_hog[i, j, k] * glyph[:, :, k])

    return image

def read_db(str_db, float_data = True):
    db = leveldb.LevelDB(str_db)
    datum = caffe_pb2.Datum()
    array = []
    label = []
    for k,v in db.RangeIter():
        dt = datum.FromString(v)
        if float_data:
          array.append(dt.float_data)
        else: 
          array.append(np.fromstring(dt.data, dtype=np.uint8))
        label.append(dt.label)
    return np.asarray(array), np.asarray(label)

def write_db(X, Y, fname):
    if os.path.exists(fname):
      shutil.rmtree(fname)
    assert X.shape[0] == Y.shape[0]
    X = X.reshape((X.shape[0], X.size/X.shape[0], 1, 1))
    db = leveldb.LevelDB(fname)

    for i in range(X.shape[0]):
      x = X[i]
      if x.ndim != 3:
        x = x.reshape((x.size,1,1))
      db.Put('{:08}'.format(i), caffe.io.array_to_datum(x, int(Y[i])).SerializeToString())
#      print(np.array2string(x, int(Y[i])))
    del db

def load_mnist(root, training):
  if training:
    data = 'train-images-idx3-ubyte'
    label = 'train-labels-idx1-ubyte'
    N = 60000
  else:
    data = 't10k-images-idx3-ubyte'
    label = 't10k-labels-idx1-ubyte'
    N = 10000
  with open(root+data, 'rb') as fin:
    fin.seek(16, os.SEEK_SET)
    X = np.fromfile(fin, dtype=np.uint8).reshape((N,28*28))
  with open(root+label, 'rb') as fin:
    fin.seek(8, os.SEEK_SET)
    Y = np.fromfile(fin, dtype=np.uint8)
  return X, Y

def make_mnist_data():
  X, Y = load_mnist('./', True)
  X_ = X.astype(np.float64)*0.02
#  X = X.astype(np.float64)*0.02
#  write_db(X, Y, 'mnist_train')

#  X_, Y_ = read_db('mnist_train', True)
#  assert np.abs((X - X_)).mean() < 1e-5
#  assert (Y != Y_).sum() == 0

  X2, Y2 = load_mnist('./', False)
  X2 = X2.astype(np.float64)*0.02
#  write_db(X2, Y2, 'mnist_test')

  X3 = np.concatenate((X,X2), axis=0)
  Y3 = np.concatenate((Y,Y2), axis=0)
#  write_db(X3,Y3, 'mnist_total')
  return X, Y

def load_stl(fname):
  num_core=multiprocessing.cpu_count()

  X = np.fromfile('./stl10_binary/'+fname, dtype=np.uint8)
  print(f'fname: {fname}')
  X = X.reshape((-1, 3, 96, 96)).transpose((0,3,2,1))
#  images = np.reshape(everything, (-1, 3, 96, 96))
#  images = np.transpose(images, (0, 3, 2, 1))
  print(f'X.shape: {X.shape}')
  global GH

  dispImg(X[:100, :, :, [2,1,0]], 10, fname+'_org.jpg')

  n_jobs = num_core # 10
  cmap_size = (8,8)
  N = X.shape[0]

  H = np.asarray(
      Parallel(n_jobs=n_jobs)\
      (delayed(hog)\
      (X[i], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2", transform_sqrt=True, feature_vector=True)
      for i in range(N))
      )
#  H = np.asarray(Parallel(n_jobs=n_jobs)(delayed(feature.hog)(X[i]) for i in range(N)))
#  for i in range(N):
#    H = hog(X[i], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2", transform_sqrt=True, feature_vector=True)

  print(f'H.shape: {H.shape}, H.size: {H.size}')
  print(f'H[0]: {H[0]}')
#  fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
#                    cells_per_block=(1, 1), visualize=True, channel_axis=-1)
  GH = H
  H_img = np.repeat(np.asarray([ hog_picture(H[i], 9) for i in range(100) ])[:, :,:,np.newaxis], 3, 3)
  dispImg(H_img, 10, fname+'_hog.jpg') 
  H = H.reshape((H.shape[0], H.size/N))

  X_small = np.asarray(Parallel(n_jobs=n_jobs)( delayed(cv2.resize)(X[i], cmap_size) for i in range(N) ))
  crcb = np.asarray(Parallel(n_jobs=n_jobs)( delayed(cv2.cvtColor)(X_small[i], cv.CV_RGB2YCrCb) for i in range(N) ))
  crcb = crcb[:,:,:,1:]
  crcb = crcb.reshape((crcb.shape[0], crcb.size/N))

  feature = np.concatenate(((H-0.2)*10.0, (crcb-128.0)/10.0), axis=1)
  print(feature.shape)

  return feature, X[:,:,:,[2,1,0]]


def make_stl_data():
  np.random.seed(1234)
  random.seed(1234)
  X_train, img_train = load_stl('train_X.bin')
  X_test, img_test = load_stl('test_X.bin')
  X_unlabel, img_unlabel = load_stl('unlabeled_X.bin')
  Y_train = np.fromfile('./stl10_binary/train_y.bin', dtype=np.uint8) - 1
  Y_test = np.fromfile('./stl10_binary/test_y.bin', dtype=np.uint8) - 1

  X_total = np.concatenate((X_train, X_test), axis=0)
  img_total = np.concatenate((img_train, img_test), axis=0)
  Y_total = np.concatenate((Y_train, Y_test))
  p = np.random.permutation(X_total.shape[0])
  X_total = X_total[p]
  img_total = img_total[p]
  Y_total = Y_total[p]
  write_db(X_total, Y_total, 'stl_total')
  write_db(img_total, Y_total, 'stl_img')

  X = np.concatenate((X_total, X_unlabel), axis=0)
  p = np.random.permutation(X.shape[0])
  X = X[p]
  Y = np.zeros((X.shape[0],))
  N = X.shape[0]*4/5
  write_db(X[:N], Y[:N], 'stl_train')
  write_db(X[N:], Y[N:], 'stl_test')

GH
print(f'GH.shape[0], int(GH.size/5000): {GH.shape[0]}, {int(GH.size/5000)}')
GH = GH.reshape((GH.shape[0], int(GH.size/5000)))
#make_mnist_data()
#GH = make_stl_data()
#for i in range(100):
#  hog_picture(GH[i], 9)



GH.shape[0], int(GH.size/5000): 5000, 4356


In [ ]:
# Your code goes here. Make sure it does not get cut off
# You can use the code below to help you get started. You're welcome to modify
# the code or remove it entirely: it's just here so that you don't get stuck
# reading files

import glob
from tqdm import tqdm

# Save numpy arrays
def save_numpy_arrays(array_save_path, train_data, test_w_data, test_m_data):
  np.savez(array_save_path, train_data = train_data, test_m_data = test_m_data, test_w_data = test_w_data)

# Load numpy arrays
def load_numpy_arrays(array_load_path):
  train_data = None
  test_m_data = None
  test_w_data = None
  data = np.load(array_load_path, allow_pickle = True)
  train_data = data['train_data']
  test_m_data = data['test_m_data']
  test_w_data = data['test_m_data']
  if DEBUG:
    print(f'data.files {data.files}')
    print(f'train_data.shape {train_data.shape}')
    print(f'test_w_data.shape {test_w_data.shape}')
    print(f'test_m_data.shape {test_m_data.shape}')
  return train_data, test_w_data, test_m_data


# Load images to numpy arrays
def load_images(path):
  images = dict()
  # find number of unique persons in data set
  person_index = -1
  active_person = None
  for file in tqdm(sorted(glob.glob(path))):
    filename = file.split("/")[-1]   # get the name of the .jpg file
    person_n, shoe, leg, _ = filename.split("_")
    if active_person != person_n: # Increment person index each time new person detected
      person_index += 1
      active_person = person_n
    if DEBUG:
      print(f'person_index: {person_index} active_person: {active_person} person_n: {person_n}')
  print(f'Total person_index: {person_index}')

  # Allocate the np array according to found number of unique persons in data set
  train_data_np = np.empty((person_index+1, 3, 2, 224, 224, 3))
  person_index = -1
  active_person = None
  for file in tqdm(sorted(glob.glob(path))):
    filename = file.split("/")[-1]   # get the name of the .jpg file
    img = plt.imread(file)           # read the image as a numpy array
    images[filename] = img[:, :, :3] # remove the alpha channel

    person_n, shoe, leg, _ = filename.split("_")
    if active_person != person_n: # Increment person index each time new person detected
      person_index += 1
      active_person = person_n
    if DEBUG:
      print(f'person_index: {person_index} active_person: {active_person} person_n: {person_n}')
    shoe_n = int(shoe)
    shoe_n -= 1                     # Start shoe index from 0
    leg = 1 if leg == 'right' else 0
    train_data_np[person_index, shoe_n, leg,:,:,:] = images[filename]
  return train_data_np

def create_and_save_data(array_save_path):
  # Load images for train, test_w and test_m data sets
  train_data = load_images(path_train)
  test_w_data = load_images(path_test_w)
  test_m_data = load_images(path_test_m)

  # Process the loaded data
  # Divide data by 255
  train_data /= 255.0
  test_w_data /= 255.0
  test_m_data /= 255.0

  # Subtract 0.5
  train_data -= 0.5
  test_w_data -= 0.5
  test_m_data -= 0.5

  # Save the data sets for future use
  save_numpy_arrays(array_save_path, train_data, test_w_data, test_m_data)
  return train_data, test_w_data, test_m_data

if DEBUG:
  sample_image = train_data_np[0,0,0,:,:,:]
  plt.figure()
  plt.imshow(sample_image)
  images[4,0,0,:,:,:]

In [ ]:
# These data sets will be initialized at the end of this cell
train_data = None
test_m_data = None
test_w_data = None
valid_set = None
train_set = None

if CREATE_DATA:
  # Create and save data sets
  print('Create numpy arrays from raw data sets and save to GDrive')
  train_data, test_w_data, test_m_data = create_and_save_data(array_save_path)
else:
  print('Load the numpy arrays from GDrive')
  # Load the data files and initiate global variables:
  # train_data, test_m_data, test_w_data, valid_set, train_set
  # Verify that data file exists
  # !ls -la '/content/gdrive/My Drive/IntroDeepLearning2022Data/ex3'
  train_data, test_w_data, test_m_data = load_numpy_arrays(array_load_path)
  # Split training data to training and validation

# Split the train data to train (80%) and validation (20%) data sets
split_idx = int(train_data.shape[0]/5)
print(f'train_data.shape: {train_data.shape}, split_idx: {split_idx}')
valid_set, train_set = train_data[:split_idx], train_data[split_idx:]
print(f'valid_set.shape: {valid_set.shape}, train_set.shape: {train_set.shape}')
#print(train_data)
#print(test_m_data)
#print(test_w_data)
#print(valid_set)
#print(train_set)



In [ ]:
# Run this code, include the image in your PDF submission
plt.figure()
plt.imshow(train_data[4,0,0,:,:,:]+ 0.5) # left shoe of first pair submitted by 5th student
plt.figure()
plt.imshow(train_data[4,0,1,:,:,:]+ 0.5) # right shoe of first pair submitted by 5th student
plt.figure()
plt.imshow(train_data[4,1,1,:,:,:]+ 0.5) # right shoe of second pair submitted by 5th student

### Part (b)

In [ ]:
# Your code goes here
def generate_same_pair(data):
  paired_data = []
  for person in data:
    for pair in person:
      paired_data.append(cv2.vconcat(pair))
  return np.array(paired_data)

# Run this code, include the result with your PDF submission
print(train_data.shape) # if this is [N, 3, 2, 224, 224, 3]
print(generate_same_pair(train_data).shape) # should be [N*3, 448, 224, 3]
plt.imshow(generate_same_pair(train_data)[0] + 0.5) # should show 2 shoes from the same pair

## Models Creation


In [ ]:
class CNN(nn.Module):
    def __init__(self, n=4, kernel_size=5, stride=2, input_shape=(3, 448, 224)):
      super(CNN, self).__init__()
      # TODO: complete this method
      padding_num = int((kernel_size-1)/2)
      self.padding = (padding_num, padding_num)
      self.C, self.H, self.W = input_shape

      self.cnn1 = torch.nn.Conv2d(in_channels=self.C, out_channels=n, stride=stride, kernel_size=kernel_size, padding=self.padding)
      self.cnn2 = torch.nn.Conv2d(in_channels=n, out_channels=2*n, stride=stride, kernel_size=kernel_size, padding=self.padding)
      self.cnn3 = torch.nn.Conv2d(in_channels=2*n, out_channels=4*n, stride=stride, kernel_size=kernel_size, padding=self.padding)
      self.cnn4 = torch.nn.Conv2d(in_channels=4*n, out_channels=8*n, stride=stride, kernel_size=kernel_size, padding=self.padding)
      self.flatten = torch.nn.Flatten()
      self.le_relu = torch.nn.LeakyReLU(0.1)
      self.relu = torch.nn.ReLU()
      self.logsoftmax = torch.nn.LogSoftmax(dim=1)

      # after 4 Conv2D with stride=(2,2), the image shape is (h/(2**4),w/(2**4)), with 8*n channels
      # thus, Linear input shape: (8*n)*((self.H*self.W)/(2**8))
      fc1_inp = int((8*n)*((self.H*self.W)/(2**8)))
      self.fc1 = torch.nn.Linear(in_features=fc1_inp, out_features=100)
      self.fc2 = torch.nn.Linear(in_features=100, out_features=2)

    # TODO: complete this class
    def forward(self, x):
      if DEBUG_NN:
        print(f'in x.shape: {x.shape:}')
      x = self.le_relu(self.cnn1(x))
      x = self.le_relu(self.cnn2(x))
      x = self.le_relu(self.cnn3(x))
      x = self.le_relu(self.cnn4(x))
      x = self.flatten(x)
      y = self.relu(self.fc1(x))
      y = self.fc2(y)
      out = self.logsoftmax(y)
      if DEBUG_NN:
         print(f'log_softmax out.shape: {out.shape:}')
      return out

### Part

<img src="https://drive.google.com/uc?id=1B59VE43X-6Dw3ag-9Ndn6vPEzbnFem8K" width="400px" />


Complete the manipulation in the `forward()` method (by slicing and using
the function `torch.cat`). The input to the first convolutional layer
should have 6 channels instead of 3 (input shape $6 \times 224 \times 224$).

In [ ]:
# function to count number of parameters
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

In [ ]:
def get_accuracy(model, data, batch_size=50):
    """Compute the model accuracy on the data set. This function returns two
    separate values: the model accuracy on the positive samples,
    and the model accuracy on the negative samples.

    Example Usage:

    >>> model = CNN() # create untrained model
    >>> pos_acc, neg_acc= get_accuracy(model, valid_data)
    >>> false_positive = 1 - pos_acc
    >>> false_negative = 1 - neg_acc
    """

    model.eval()
    n = data.shape[0]

    data_pos = generate_same_pair(data)      # should have shape [n * 3, 448, 224, 3]
    data_neg = generate_different_pair(data) # should have shape [n * 3, 448, 224, 3]

    pos_correct = 0
    for i in range(0, len(data_pos), batch_size):
        xs = torch.Tensor(data_pos[i:i+batch_size])
        # reshape (our modification)
        N, H, W, C = xs.shape
        xs = xs.view(N,C,H,W)

        zs = model(xs)
        pred = zs.max(1, keepdim=True)[1] # get the index of the max logit
        pred = pred.detach().numpy()
        pos_correct += (pred == 1).sum()
    
    neg_correct = 0
    for i in range(0, len(data_neg), batch_size):
        xs = torch.Tensor(data_neg[i:i+batch_size])
        # reshape (our modification)
        N, H, W, C = xs.shape
        xs = xs.view(N,C,H,W)

        zs = model(xs)
        pred = zs.max(1, keepdim=True)[1] # get the index of the max logit
        pred = pred.detach().numpy()
        neg_correct += (pred == 0).sum()

    return pos_correct / (n * 3), neg_correct / (n * 3)

## Training

In [ ]:
# Write your code here
def train_model(model,
                train_data=train_set,
                validation_data=valid_set,
                batch_size=100,
                num_epochs=100,
                learning_rate=0.001,
                weight_decay=0,
                checkpoint_path=None):
  
    # choice of loss function & optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),
                           lr=learning_rate,
                           weight_decay=weight_decay)

    iters, losses = [], []
    epocs_vec, train_accs_pos, train_accs_neg, val_accs_pos, val_accs_neg  = [], [] ,[], [], []

    # obtaining the positive and negative samples
    pos_train = generate_same_pair(train_data)
    neg_train = generate_different_pair(train_data)

    # labels (one-hot representation)
    pos_labels = torch.Tensor(np.array([np.eye(2)[1] for _ in range(batch_size // 2)]))  # one_hot(pos = 1) <=> [0,1]
    neg_labels = torch.Tensor(np.array([np.eye(2)[0] for _ in range(batch_size // 2)]))  # one_hot(neg = 0) <=> [1,0]
    labels = torch.cat([pos_labels, neg_labels], dim=0)

    data_size = len(pos_train) + len(neg_train)

    # main training loop
    n = 0 # total number of iterations

    for epoch in range(num_epochs):  # epochs
        for i in range(0, data_size, batch_size):  # iterations in each epoch
           
            if (i + batch_size) > data_size:
                break

            # shuffle data (assume len(pos_train)=len(neg_train))
            reindex = np.random.permutation(len(pos_train))
            pos_train = pos_train[reindex]
            reindex = np.random.permutation(len(pos_train))
            neg_train = neg_train[reindex]

            # create batch
            pos_batch = pos_train[: batch_size // 2]
            neg_batch = neg_train[: batch_size // 2]

            # convert to tensors
            pos_batch = torch.Tensor(pos_batch)
            neg_batch = torch.Tensor(neg_batch)

            # reshape
            N, H, W, C = pos_batch.shape
            pos_batch = pos_batch.view(N,C,H,W)
            neg_batch = neg_batch.view(N,C,H,W)

            # create input tensor
            input_batch = torch.cat([pos_batch, neg_batch], dim=0)

            # train
            optimizer.zero_grad()         # a clean up step for PyTorch
            y = model(input_batch)        # compute prediction logit
            loss = criterion(y, labels)   # compute the total loss
            loss.backward()               # compute updates for each parameter
            optimizer.step()              # make the updates for each parameter

            # save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)  # compute *average* loss

            # increment the iteration number
            n += 1


        epocs_vec.append(epoch)
        train_cost = float(loss.detach().numpy())

        pos_acc_train, neg_acc_train = get_accuracy(model, train_data, batch_size=batch_size)
        train_accs_pos.append(pos_acc_train)
        train_accs_neg.append(neg_acc_train)

        pos_acc_val, neg_acc_val = get_accuracy(model, valid_set, batch_size=batch_size)
        val_accs_pos.append(pos_acc_val)
        val_accs_neg.append(neg_acc_val)      
    
        print("Epoch %d, Iter %d. [Val Acc %.0f%% (pos: %.0f%%, neg: %.0f%%)] [Train Acc %.0f%% (pos: %.0f%%, neg: %.0f%%), Loss %f]" % (
              epoch, n, (pos_acc_val + neg_acc_val) * 50, pos_acc_val * 100, neg_acc_val * 100, (pos_acc_train + neg_acc_train) * 50, pos_acc_train * 100, neg_acc_train * 100, train_cost))

        if (checkpoint_path is not None) and n > 0:
            torch.save(model.state_dict(), checkpoint_path.format(epoch))

    return iters, losses, epocs_vec, np.array(train_accs_pos), np.array(train_accs_neg), np.array(val_accs_pos), np.array(val_accs_neg)

In [ ]:
def plot_learning_curve(iters, losses, epocs_vec, train_accs_pos, train_accs_neg, val_accs_pos, val_accs_neg):
    """
    Plot the learning curve.
    """
    plt.title("Learning Curve: Loss per Iteration")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    fig, axs = plt.subplots(1, 3, figsize=(15,5))
    fig.suptitle("Learning Curve: Accuracy per Epoch")

    axs[0].plot(epocs_vec, train_accs_pos, label="Train")
    axs[0].plot(epocs_vec, val_accs_pos, label="Validation")
    axs[0].set_title('Positive Samples')
    axs[0].set_xlabel("Epochs")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend()

    axs[1].plot(epocs_vec, train_accs_neg, label="Train")
    axs[1].plot(epocs_vec, val_accs_neg, label="Validation")
    axs[1].set_title('Negative Samples')
    axs[1].set_xlabel("Epochs")
    axs[1].set_ylabel("Accuracy")
    axs[1].legend()

    axs[2].plot(epocs_vec, (train_accs_neg+train_accs_pos)/2, label="Train")
    axs[2].plot(epocs_vec, (val_accs_neg+val_accs_pos)/2, label="Validation")
    axs[2].set_title('Total Accuracy')
    axs[2].set_xlabel("Epochs")
    axs[2].set_ylabel("Accuracy")
    axs[2].legend()
    

In [ ]:
toy_data_size = 6

toy_train_data = train_set[:toy_data_size]
toy_valid_data = valid_set[:toy_data_size]

print("=== CNN ===")
cnn = CNN()
cnn_learning_curve_info = train_model(model=cnn,
                                  train_data=toy_train_data,
                                  validation_data=toy_valid_data,
                                  batch_size=6*toy_data_size,
                                  num_epochs=20,
                                  learning_rate=2e-3,
                                  weight_decay=0,
                                  checkpoint_path=None)

plot_learning_curve(*cnn_learning_curve_info)
print(f'len(cnn_learning_curve_info): {len(cnn_learning_curve_info)}')
print(f'number of parememters cnn: {get_n_params(cnn)}')


In [ ]:
print("=== CNNChannel ===")

cnnC = CNNChannel(n=4, kernel_size=5)
cnnc_learning_curve_info = train_model(model=cnnC,
                                  train_data=train_data,
                                  validation_data=valid_set,
                                  batch_size=80,
                                  num_epochs=20,
                                  learning_rate=1e-3,
                                  weight_decay=0,
                                  checkpoint_path = checkpoint_path)

plot_learning_curve(*cnnc_learning_curve_info)

## Testing

In [ ]:
# Write your code here. Make sure to include the test accuracy in your report

# load CNNChannel model.
# Model from last learning iteration had the best learning results.
# We use model from iteration 20.
model = CNNChannel()
model.load_state_dict(torch.load(best_model_path))

test_m_acc = get_accuracy(model=model, data=test_m_data, batch_size=100)
test_m_acc = np.array(test_m_acc) * 100
tot_acc_test_m = np.mean(test_m_acc)

test_w_acc =get_accuracy(model=model, data=test_w_data, batch_size=100)
test_w_acc = np.array(test_w_acc) * 100
tot_acc_test_w = np.mean(test_w_acc)

print('Man Shoes   [test_m]: Total accuracy = {:.2f}%, pos accuracy = {:.2f}%, neg accuracy = {:.2f}%'.format(tot_acc_test_m, test_m_acc[0], test_m_acc[1]))
print('Woman Shoes [test_w]: Total accuracy = {:.2f}%, pos accuracy = {:.2f}%, neg accuracy = {:.2f}%'.format(tot_acc_test_w, test_w_acc[0], test_w_acc[1]))

In [ ]:
def get_prediction(model, input):
    """ returns the predicion of a given model
      inspired by the get_accuracy function
    """

    model.eval()
    xs = torch.Tensor(input)
    N, H, W, C = xs.shape
    xs = xs.view(N,C,H,W)
    zs = model(xs)
    pred = zs.max(1, keepdim=True)[1] # get the index of the max logit
    pred = pred.detach().numpy()[0,0]
    return pred

def get_correct_incorrect_pairs(data, gender):
    same_pair = generate_same_pair(data)
    found_true_positive = False
    found_false_positive = False
    for im in same_pair:
        inp = np.expand_dims(im, 0)
        pred = get_prediction(model, inp)
        if pred == 1 and not found_true_positive:
            found_true_positive = True
            plt.figure()
            plt.title(gender + ' test data CORRECT classification')
            plt.imshow(im + 0.5)
        if pred == 0 and not found_false_positive:
            #found_false_positive = True
            plt.figure()
            plt.title(gender + ' test data INCORRECT classification')
            plt.imshow(im + 0.5)
        if found_true_positive and found_false_positive:
            break

In [ ]:
# As we can see from the negative results, our model sometimes makes wrong decisions and recognizes the shoes from the same pair as from different pairs.
# We also see that pair number 4 contains different shoes - probably this is data set issue.
get_correct_incorrect_pairs(test_m_data, 'Man')

# Automatic PDF Generation and store in GDrive

In [ ]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc > /dev/null 2>&1
!pip install pypandoc > /dev/null 2>&1

from google.colab import drive
drive.mount('/content/drive')
!cp 'drive/My Drive/Colab Notebooks/Assignment3_work.ipynb' ./

!jupyter nbconvert --to PDF "Assignment3_work.ipynb" > /dev/null 2>&1
!echo "pdf file generated"
#!ls -la
!cp './Assignment3_work.pdf' 'drive/My Drive/Colab Notebooks'